In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('default')
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_squared_error
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
import joblib
import pickle
import math
import time
from collections import defaultdict
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

In [ ]:
import sys
sys.path.append('./model')
from models import BayesianNeuralNetwork, train_bnn, predict_bnn
from models import train_gpr, predict_gpr

## Define Function

In [ ]:
def preprocess_inputdata(all_data):
  NUM_OF_BINS = 502
  input_data = []
  output = []
  errors = []
  z_data = []

  #exlclude_paras = {"c": ["0.25", "0.75", "1.25", "1.75"]}
  exlclude_paras = {}
  for key, data in all_data.items():
    #print(key, data)
    density_profiles = []
    density_errors = []
    z_data_values = []
    input_names = key.split("_")[0::2]
    input_paras = key.split("_")[1::2]

    ignore_this = False
    for key_p, params in exlclude_paras.items():
        if input_paras[input_names.index(key_p)] in params:
            ignore_this= True
            break
    if ignore_this:
        continue

    input_data.append(input_paras)
    density_profiles.append(data['pos'][:,1])
    density_profiles.append(data['neg'][:,1])
    output.append(density_profiles)
    density_errors.append(data['pos'][:,2])
    density_errors.append(data['neg'][:,2])
    errors.append(density_errors)
    z_data_values.append(data['pos'][:,0])
    z_data_values.append(data['neg'][:,0])
    z_data.append(z_data_values)

    #break

  input_data = np.array(input_data)
  output = np.array(output).reshape(-1,NUM_OF_BINS*2)
  errors = np.array(errors).reshape(-1,NUM_OF_BINS*2)
  z_data = np.array(z_data).reshape(-1,NUM_OF_BINS*2)
  print("Input data shape: {}".format(input_data.shape))
  print("Output data shape: {}".format(output.shape))
  print("error bar data shape: {}".format(errors.shape))
  print("Bin center data shape: {}".format(z_data.shape))

  return input_data, output, errors, z_data


def compute_peak_density(input_data, output, errors, z_data):
    output_peak_density = np.zeros((input_data.shape[0], 1))
    error_peak_density = np.zeros((input_data.shape[0], 1))
    for i in range(input_data.shape[0]):
        max_index = np.argmax(output[i])
        output_peak_density[i] = output[i][max_index]
        error_peak_density[i] = errors[i][max_index]
    return output_peak_density, error_peak_density

def calculate_rmse(actual_values, predicted_values):
    mse = mean_squared_error(actual_values, predicted_values)
    rmse = np.sqrt(mse)
    return rmse

In [ ]:
#active learning loop for GPR

def active_learning_loop_GPR(xtrain, ytrain, xtest, ytest, xremain, yremain,
                         kernel_variance, kernel_lengthscale, 
                         white_kernel_variance, max_iterations, epochs, n_samples=3):
    # Initialize current datasets
    current_xtrain = xtrain.copy()
    current_ytrain = ytrain.copy()
    current_xtest = xtest.copy()
    current_ytest = ytest.copy()
    current_xremain = xremain.copy()
    current_yremain = yremain.copy()
    
    # Track metrics
    metrics = {
        'rmse': [],
        'training_size': [],
        'test_size': [],
        'std': []
    }
    
    for epoch in range(epochs+1):
        print(f"\n--- Active Learning Epoch {epoch}/{epochs} ---")
        print(f"Training samples: {len(current_xtrain)}, Test samples: {len(current_xtest)}")
        
        # Train GPR model
        start_time = time.time()
        model = train_gpr(current_xtrain, current_ytrain, 
                          kernel_variance, kernel_lengthscale,
                          white_kernel_variance, max_iterations)
        train_time = time.time() - start_time
        
        # Predict on current test set
        mean, std = predict_gpr(model, current_xtest)
        
        # Calculate RMSE
        rmse = calculate_rmse(current_ytest, mean)
        metrics['rmse'].append(rmse)
        metrics['training_size'].append(len(current_xtrain))
        metrics['test_size'].append(len(current_xtest))
        metrics['std'].append(np.mean(std))
        
        print(f"Training time: {train_time:.2f} seconds | RMSE: {rmse:.4f}")
        print(f"Prediction stats: Mean={np.mean(mean):.4f} ± Std={np.mean(std):.4f}")

        #Evaluate on the remaining dataset pool
        mean_remain, std_remain = predict_gpr(model, current_xremain)
       
        # Break if remain set is exhausted
        if len(current_xremain) <= n_samples:
            print("Remain set exhausted. Stopping early.")
            break
        
        # Identify top N uncertain samples (highest std)
        std_flattened = std_remain.flatten()
        topN_indices = np.argsort(std_flattened)[-n_samples:]
        
        # Add to training set
        current_xtrain = np.vstack([current_xtrain, current_xremain[topN_indices]])
        current_ytrain = np.vstack([current_ytrain, current_yremain[topN_indices]])
        
        # Remove from remain set
        mask = np.ones(len(current_xremain), dtype=bool)
        mask[topN_indices] = False
        current_xremain = current_xremain[mask]
        current_yremain = current_yremain[mask]

    
    return metrics, model

In [ ]:
#active learning loop for BNN

def active_learning_loop_BNN(xtrain, ytrain, xtest, ytest, xremain, yremain,
                         n_features, hidden_layers, weight_init_std, 
                         log_std_init_mean, log_std_init_std, log_std_clamp,
                         bnn_epochs, learning_rate, grad_clip_norm,
                         al_epochs, n_samples=5, n_mc_samples=100, batch_size=32):
    # Convert to PyTorch tensors
    xtrain_t = torch.FloatTensor(xtrain)
    ytrain_t = torch.FloatTensor(ytrain)
    xtest_t = torch.FloatTensor(xtest)
    ytest_t = torch.FloatTensor(ytest)
    xremain_t = torch.FloatTensor(xremain)
    yremain_t = torch.FloatTensor(yremain)
    
    # Initialize current datasets
    current_xtrain = xtrain_t.clone()
    current_ytrain = ytrain_t.clone()
    current_xtest = xtest_t.clone()
    current_ytest = ytest_t.clone()
    current_xremain = xremain_t.clone()
    current_yremain = yremain_t.clone()
    
    # Track metrics
    metrics = {
        'rmse': [],
        'training_size': [],
        'test_size': [],
        'std': []
    }
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    for epoch in range(al_epochs+1):
        print(f"\n--- Active Learning Epoch {epoch}/{al_epochs} ---")
        print(f"Training samples: {len(current_xtrain)}, Test samples: {len(current_xtest)}")
        
        # Create DataLoader
        train_dataset = TensorDataset(current_xtrain, current_ytrain)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        
        # Initialize and train BNN
        model = BayesianNeuralNetwork(
            n_features, hidden_layers, weight_init_std,
            log_std_init_mean, log_std_init_std, log_std_clamp
        ).to(device)
        
        start_time = time.time()
        train_bnn(model, train_loader, bnn_epochs, learning_rate, grad_clip_norm)
        train_time = time.time() - start_time
        
        # Predict on test set
        model.eval()
        with torch.no_grad():
            test_preds, test_std = predict_bnn(model, current_xtest.to(device), n_samples=n_mc_samples)
        
        # Calculate RMSE
        rmse = calculate_rmse(current_ytest.numpy(), test_preds.cpu().numpy())
        metrics['rmse'].append(rmse)
        metrics['training_size'].append(len(current_xtrain))
        metrics['test_size'].append(len(current_xtest))
        metrics['std'].append(test_std.mean().item())
        
        print(f"Training time: {train_time:.2f} seconds | RMSE: {rmse:.4f}")
        print(f"Prediction stats: Mean={test_preds.mean().item():.4f} ± Std={test_std.mean().item():.4f}")

        # Break if remain set is exhausted
        if len(current_xremain) <= n_samples:
            print("Remain set exhausted.")
            break
            
        # Predict uncertainty on remaining pool
        with torch.no_grad():
            _, remain_std = predict_bnn(model, current_xremain.to(device), n_samples=n_mc_samples)
        
        # Identify top N uncertain samples
        std_np = remain_std.cpu().numpy()
        topN_indices = np.argsort(std_np)[-n_samples:]
        
        # Add to training set
        current_xtrain = torch.cat([current_xtrain, current_xremain[topN_indices]])
        current_ytrain = torch.cat([current_ytrain, current_yremain[topN_indices]])
        
        # Remove from remain set
        mask = torch.ones(len(current_xremain), dtype=torch.bool)
        mask[topN_indices] = False
        current_xremain = current_xremain[mask]
        current_yremain = current_yremain[mask]

    return metrics, model


In [ ]:
#active learning for MCD
def active_learning_loop_MCD(xtrain, ytrain, xtest, ytest, xremain, yremain,
                             n_features, hidden_units, dropout_rate,
                             mcd_epochs, learning_rate, al_epochs,
                             n_samples=5, T=100, batch_size=32):
    # Convert to PyTorch tensors
    xtrain_t = torch.FloatTensor(xtrain)
    ytrain_t = torch.FloatTensor(ytrain)
    xtest_t = torch.FloatTensor(xtest)
    ytest_t = torch.FloatTensor(ytest)
    xremain_t = torch.FloatTensor(xremain)
    yremain_t = torch.FloatTensor(yremain)
    
    # Initialize current datasets
    current_xtrain = xtrain_t.clone()
    current_ytrain = ytrain_t.clone()
    current_xtest = xtest_t.clone()
    current_ytest = ytest_t.clone()
    current_xremain = xremain_t.clone()
    current_yremain = yremain_t.clone()
    
    # Track metrics
    metrics = {
        'rmse': [],
        'training_size': [],
        'test_size': [],
        'std': []
    }
    
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Create test DataLoader for prediction
    test_dataset = TensorDataset(current_xtest, current_ytest)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    # Create remain DataLoader for uncertainty sampling
    remain_dataset = TensorDataset(current_xremain, current_yremain)
    remain_loader = DataLoader(remain_dataset, batch_size=batch_size, shuffle=False)
    
    for epoch in range(al_epochs + 1):
        print(f"\n--- Active Learning Epoch {epoch}/{al_epochs} ---")
        print(f"Training samples: {len(current_xtrain)}, Remain pool: {len(current_xremain)}")
        
        # Create DataLoader for current training set
        train_dataset = TensorDataset(current_xtrain, current_ytrain)
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
        
        # Initialize MCD model
        model = DropoutModel(
            n_features, 
            hidden_units[0], 
            hidden_units[1] if len(hidden_units) > 1 else hidden_units[0],
            hidden_units[2] if len(hidden_units) > 2 else hidden_units[0],
            dropout_rate
        ).to(device)
        
        # Train MCD model
        start_time = time.time()
        train_mcd(model, train_loader, mcd_epochs, learning_rate)
        train_time = time.time() - start_time
        
        # Predict on test set
        test_mean, test_std = predict_mcd(model, test_loader, T)
        
        # Calculate RMSE
        rmse = calculate_rmse(current_ytest.numpy(), test_mean)
        metrics['rmse'].append(rmse)
        metrics['training_size'].append(len(current_xtrain))
        metrics['test_size'].append(len(current_xtest))
        metrics['std'].append(test_std.mean())
        
        print(f"Training time: {train_time:.2f} seconds | RMSE: {rmse:.6f}")
        print(f"Prediction stats: Mean={test_mean.mean():.6f} ± Std={test_std.mean():.6f}")

        # Break if remain set is exhausted
        if len(current_xremain) <= n_samples or epoch == al_epochs:
            print("Remain pool exhausted or final epoch reached")
            break
            
        # Predict uncertainty on remaining pool
        remain_mean, remain_std = predict_mcd(model, remain_loader, T)
        
        # Identify top N uncertain samples
        topN_indices = np.argsort(remain_std)[-n_samples:]
        
        # Add to training set
        current_xtrain = torch.cat([current_xtrain, current_xremain[topN_indices]])
        current_ytrain = torch.cat([current_ytrain, current_yremain[topN_indices]])
        
        # Remove from remain set
        mask = torch.ones(len(current_xremain), dtype=torch.bool)
        mask[topN_indices] = False
        current_xremain = current_xremain[mask]
        current_yremain = current_yremain[mask]
        
        # Update remain DataLoader
        remain_dataset = TensorDataset(current_xremain, current_yremain)
        remain_loader = DataLoader(remain_dataset, batch_size=batch_size, shuffle=False)

    return metrics, model

## Preprocess data

In [ ]:
file_path= 'data/'
with open(file_path+ 'data_dump_density_preprocessed_train.pk', 'rb') as handle:
    processed_all_data_preprocessed_train = pickle.load(handle)
with open(file_path+ 'data_dump_density_preprocessed_test.pk', 'rb') as handle:
    processed_all_data_preprocessed_test = pickle.load(handle)

#reduce training set size by randomly excluding N data.
np.random.seed(0)   #fix random seed
index_ = np.random.choice(len(processed_all_data_preprocessed_train.keys()), 3500,replace=False)  #change choice size to select reduced training samples
excluded_index_ = np.delete(np.arange(0,len(processed_all_data_preprocessed_train.keys())), index_)
train_ = {}
exclude_ = {}
for index in index_:
    exclude_[list(processed_all_data_preprocessed_train.keys())[index]] = processed_all_data_preprocessed_train[list(processed_all_data_preprocessed_train.keys())[index]]
for index in excluded_index_:
    train_[list(processed_all_data_preprocessed_train.keys())[index]] = processed_all_data_preprocessed_train[list(processed_all_data_preprocessed_train.keys())[index]]


#Preprocess data to density output (NX1004)
input_data, output, errors, z_data = preprocess_inputdata(train_)
input_data_remain, output_remain, errors_remain, z_data_remain = preprocess_inputdata(exclude_)
input_data_test, output_test_raw, errors_test_raw, z_data_test = preprocess_inputdata(processed_all_data_preprocessed_test)


#Covert to peak density output (NX1)
output_train, errors_train = compute_peak_density(input_data, output, errors, z_data)
output_train_remain, errors_train_remain = compute_peak_density(input_data_remain, output_remain, errors_remain, z_data_remain)
output_test, errors_test = compute_peak_density(input_data_test, output_test_raw, errors_test_raw, z_data_test)

In [ ]:
#cross validation 
#split ranges 0.8 to 1
train_test_split = 1
seed = 65231


input_data_suff, output_suff,  errors_suff, z_data_shuff = shuffle(input_data, output_train, errors_train, z_data, random_state=seed)
#input_data_suff, output_suff,  errors_suff, z_data_shuff = shuffle(input_data, output[:, :100], errors[:, :100], z_data[:, :100], random_state=seed)

train_test_split_ = int(input_data_suff.shape[0]*train_test_split)

x_train = input_data_suff[0:train_test_split_]#.astype("float64")
x_test = input_data_suff[train_test_split_:]#.astype("float64")


y_train = output_suff[0:train_test_split_]#.astype("float64")
y_test = output_suff[train_test_split_:]#.astype("float64")


error_train = errors_suff[0:train_test_split_]#.astype("float64")
error_test = errors_suff[train_test_split_:]#.astype("float64")

z_data_train = z_data_shuff[0:train_test_split_]#.astype("float64")
z_data_test = z_data_shuff[train_test_split_:]#.astype("float64")


#x_train, x_test, y_train, y_test = spliter.train_test_split(input_data, output, test_size=(1-train_test_split), random_state=100)

print("Train input: ", x_train.shape)
print("Train Output", y_train.shape)
print("Test input: ", x_test.shape)
print("Test Output", y_test.shape)

In [ ]:
scaler = joblib.load(file_path+'scaler_new.pkl')
scaled_x_train = scaler.transform(x_train)
scaled_x_test = scaler.transform(input_data_test)


scaler_y = joblib.load(file_path+'minmax_scaler_peak_label.joblib')
scaler_error = joblib.load(file_path+'minmax_scaler_peak_error.joblib')

scaled_y_train = scaler_y.transform(y_train)
scaled_y_test = scaler_y.transform(output_test)
scaled_error_train =  scaler_error.transform(error_train)
scaled_error_test =  scaler_error.transform(errors_test)


scaled_x_remain = scaler.transform(input_data_remain)
scaled_y_remain = scaler_y.transform(output_train_remain)
scaled_error_remain = scaler_error.transform(errors_train_remain)

## GPR

In [ ]:
KERNEL_VARIANCE = 1
KERNEL_LENGTHSCALE = 1
WHITE_KERNEL_VARIANCE = 1
MAX_ITERATIONS = 2000
EPOCHS = 100
N_SAMPLES = 5  # Number of samples to add per epoch

# Run active learning
metrics, final_model = active_learning_loop_GPR(
    scaled_x_train, scaled_y_train,
    scaled_x_test, scaled_y_test,
    scaled_x_remain, scaled_y_remain,
    KERNEL_VARIANCE, KERNEL_LENGTHSCALE,
    WHITE_KERNEL_VARIANCE, MAX_ITERATIONS, 
    EPOCHS, N_SAMPLES
)

# Print summary
print("\n=== Active Learning Summary ===")
print(f"Initial Training Size: {len(scaled_x_train)}")
print(f"Final Training Size: {metrics['training_size'][-1]}")
print(f"Final Test Size: {metrics['test_size'][-1]}")
print(f"Minimum RMSE: {min(metrics['rmse']):.4f}")

In [ ]:
training_set = [50, 100, 150, 200, 250, 300, 400, 550]
RMSE = [0.0287, 0.0194,0.0152, 0.0122, 0.0101, 0.0093, 0.0074, 0.0062 ]

plt.figure(figsize=(10, 6))
plt.plot(metrics['training_size'], metrics['rmse'], 
         marker='o', markersize=6, 
         linestyle='-', color='red', 
         label='Active Learning', zorder=2)
plt.plot(training_set, RMSE, 
         marker='s', markersize=6, 
         linestyle='-', color='blue', 
         label='Random Drop', zorder=4)

plt.axhline(y=0.0062, color='k', linestyle='--')

plt.xlabel('Training Size', fontsize=16)
plt.ylabel('RMSE on Testing Set', fontsize=16)
plt.title('Active Learning vs Random Droping', fontsize=14)

# Add legend and grid
plt.legend(fontsize=16)
plt.grid(True, linestyle='--', alpha=0.7)

# Adjust layout and show plot
plt.tight_layout()

#plt.savefig('al.png', dpi=300, bbox_inches='tight')
plt.show()

## BNN

In [ ]:
HIDDEN_LAYERS = [50, 50]
WEIGHT_INIT_STD = 0.1
LOG_STD_INIT_MEAN = -3
LOG_STD_INIT_STD = 0.1
LOG_STD_CLAMP = (-5, 2)
BNN_EPOCHS = 500
LEARNING_RATE = 0.01
GRAD_CLIP_NORM = 1.0
AL_EPOCHS = 10
N_SAMPLES = 5
N_MC_SAMPLES = 100

# Run active learning
metrics_bnn, final_model_bnn = active_learning_loop_BNN(
    scaled_x_train, scaled_y_train,
    scaled_x_test, scaled_y_test,
    scaled_x_remain, scaled_y_remain,
    n_features=scaled_x_train.shape[1],
    hidden_layers=HIDDEN_LAYERS,
    weight_init_std=WEIGHT_INIT_STD,
    log_std_init_mean=LOG_STD_INIT_MEAN,
    log_std_init_std=LOG_STD_INIT_STD,
    log_std_clamp=LOG_STD_CLAMP,
    bnn_epochs=BNN_EPOCHS,
    learning_rate=LEARNING_RATE,
    grad_clip_norm=GRAD_CLIP_NORM,
    al_epochs=AL_EPOCHS,
    n_samples=N_SAMPLES,
    n_mc_samples=N_MC_SAMPLES
)

# Print summary
print("\n=== Active Learning Summary ===")
print(f"Initial Training Size: {len(scaled_x_train)}")
print(f"Final Training Size: {metrics_bnn['training_size'][-1]}")
print(f"Final Test Size: {metrics_bnn['test_size'][-1]}")
print(f"Minimum RMSE: {min(metrics_bnn['rmse']):.4f}")

In [ ]:
training_set = []
RMSE = []

plt.figure(figsize=(10, 6))
plt.plot(metrics_bnn['training_size'], metrics_bnn['rmse'], 
         marker='o', markersize=6, 
         linestyle='-', color='red', 
         label='Active Learning', zorder=2)
plt.plot(training_set, RMSE, 
         marker='s', markersize=6, 
         linestyle='-', color='blue', 
         label='Random Drop', zorder=4)

plt.axhline(y=0, color='k', linestyle='--')

plt.xlabel('Training Size', fontsize=16)
plt.ylabel('RMSE on Testing Set', fontsize=16)
plt.title('Active Learning vs Random Droping', fontsize=14)

# Add legend and grid
plt.legend(fontsize=16)
plt.grid(True, linestyle='--', alpha=0.7)

# Adjust layout and show plot
plt.tight_layout()

#plt.savefig('al.png', dpi=300, bbox_inches='tight')
plt.show()

## MCD

In [ ]:
# Define parameters
HIDDEN_UNITS = [256, 512, 128]  # 3 hidden layers
DROPOUT_RATE = 0.2
MCD_EPOCHS = 1000
AL_EPOCHS = 10

# Run active learning
metrics_mcd, mcd_model = active_learning_loop_MCD(
    scaled_x_train, scaled_y_train,
    scaled_x_test, scaled_y_test,
    scaled_x_remain, scaled_y_remain,
    n_features=scaled_x_train.shape[1],
    hidden_units=HIDDEN_UNITS,
    dropout_rate=DROPOUT_RATE,
    mcd_epochs=MCD_EPOCHS,
    learning_rate=0.001,
    al_epochs=AL_EPOCHS,
    n_samples=5,
    T=100)

In [ ]:
training_set = []
RMSE = []

plt.figure(figsize=(10, 6))
plt.plot(metrics_mcd['training_size'], metrics_mcd['rmse'], 
         marker='o', markersize=6, 
         linestyle='-', color='red', 
         label='Active Learning', zorder=2)
plt.plot(training_set, RMSE, 
         marker='s', markersize=6, 
         linestyle='-', color='blue', 
         label='Random Drop', zorder=4)

plt.axhline(y=0, color='k', linestyle='--')

plt.xlabel('Training Size', fontsize=16)
plt.ylabel('RMSE on Testing Set', fontsize=16)
plt.title('Active Learning vs Random Droping', fontsize=14)

# Add legend and grid
plt.legend(fontsize=16)
plt.grid(True, linestyle='--', alpha=0.7)

# Adjust layout and show plot
plt.tight_layout()

#plt.savefig('al.png', dpi=300, bbox_inches='tight')
plt.show()